In [ ]:
import numpy as np
import os
import glob 
import time 
import pandas as pd
import shutil
import re
import mdtraj as md
import MDAnalysis as mda
import pickle
import pickle as p 
import csv
import sys
import subprocess
import xml.etree.ElementTree as ET
import requests
import concurrent.futures
import multiprocessing
import nglview as nv
import h5py
import matplotlib as mpl
import matplotlib.patheffects as path_effects
import matplotlib.patches as patches
import html
import biobox as bb
import tempfile
import matplotlib.pyplot as plt
import molearn
import torch
import glob as glob
import MDAnalysis.analysis.rms as rms
import seaborn as sns
import pickle
import networkx as nx
sys.path.insert(0, os.path.join(os.path.abspath(os.pardir),'src'))

from Bio import PDB
from tqdm import tqdm
from time import time as t
from urllib.request import urlretrieve as download
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Blast.Applications import NcbipsiblastCommandline
from collections import defaultdict
from Bio.PDB import PPBuilder
from Bio.SeqUtils import seq1
from modeller import *
from modeller.automodel import *
from Bio.PDB import PDBParser
from MDAnalysis.analysis import align
from mpi4py import MPI
from glob import glob
from pprint import pprint as pp
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from glob import glob as g
from tqdm.notebook import tqdm
from collections import Counter
from matplotlib.ticker import FuncFormatter
from collections import defaultdict
from pymol import cmd
from scipy.interpolate import interp1d
from mpl_toolkits import mplot3d
from scipy import stats
from sklearn.cluster import AgglomerativeClustering, SpectralClustering
from matplotlib.colors import BoundaryNorm
from numpy.linalg import norm
from molearn.data import PDBData
from molearn.trainers import Trainer
from molearn.models.small_foldingnet import Small_AutoEncoder
from molearn.analysis.analyser import MolearnAnalysis
from copy import deepcopy
from molearn.analysis import MolearnGUI
from scipy.ndimage import rotate
from numpy.linalg import inv
from sklearn.cluster import HDBSCAN
from sklearn.metrics import silhouette_score
from itertools import combinations
from urllib.request import urlretrieve
from urllib.error import URLError
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.inspection import permutation_importance
from sklearn.tree import export_graphviz
from sklearn import tree
from scipy.stats import pearsonr


In [ ]:
"""
This script performs PCA-based dimensionality reduction on a set of protein structures and visualizes residue-level loadings and sample clustering. It supports hierarchical clustering in the PC1–PC2 space and highlights outliers or specific groups if needed.

"""
# Create a directory if it doesn't already exist
def ifnotmake(dir_path):
    if not os.path.isdir(dir_path):
        os.makedirs(dir_path)
    return dir_path

# Save cluster labels and structure names into a CSV file
def save_cluster_labels(cluster_labels, structure_names, output_filename):
    """Save cluster labels along with corresponding structure names to a CSV file."""
    import pandas as pd
    
    df = pd.DataFrame({
        'structure': structure_names,
        'cluster': cluster_labels
    })
    
    df.to_csv(output_filename, index=False)
    print(f"Saved cluster labels to {output_filename}")

# Extract the PDB ID (file name without extension) from a file path
def pdb_id(fp):
    return os.path.splitext(os.path.basename(fp))[0]

# Get a list of all unique residue names from PDB files in a folder
def braf_res(folder):
    pdb_files = glob(os.path.join(folder, "*.pdb"))
    filtered_pdb_files = [fp for fp in pdb_files if "combined" not in os.path.basename(fp)]
    if not filtered_pdb_files:
        raise IOError(f"No .pdb files found in '{folder}' (after excluding 'combined' files).")
    
    all_residues = set()
    for fp in filtered_pdb_files:
        traj = md.load(fp)
        top = traj.topology
        for res in top.residues:
            all_residues.add(res.name)
    return sorted(list(all_residues))

# Load a list of PDB files and perform PCA on the concatenated coordinates
def PCA_on_files(file_list, n_components=4):

    if len(file_list) == 0:
        raise ValueError("No files provided for PCA (check filtering).")
    print("Number of PDB files to load:", len(file_list))
    for f in file_list:
        print("  ", os.path.basename(f))

    # Concatenate all trajectories
    traj = md.join([md.load(f) for f in file_list])

    xyz = traj.xyz.reshape(-1, 3 * traj.n_atoms)

    # Perform PCA
    pca = PCA(n_components=n_components)
    proj = pca.fit_transform(xyz)
    return pca, proj

# Plot PCA component scores as residue-wise loading magnitudes
def plot_scores(cur_pca, n_comps=4, folder_for_residues=None, figure_title="PCA scores"):

    exp_var_ratio = cur_pca.explained_variance_ratio_
    print("Explained variance ratio:", exp_var_ratio)
    scores = cur_pca.components_.reshape(-1, 27, 3)[:n_comps]

    x_labels = braf_res(folder_for_residues)
    x_vals = list(range(len(scores[0])))
    col = ["red", "blue", "green", "yellow"]
    exp_var = cur_pca.explained_variance_

    # Create one subplot per component
    fig, axes = plt.subplots(n_comps, sharex=True, figsize=(8, 8))
    if n_comps == 1:
        axes = [axes]  

    for i in range(n_comps):
        # Project component vector with variance weighting
        load_val = norm(scores[i] * exp_var[i], axis=1)
        axes[i].plot(x_vals, load_val, marker=".", c=col[i % len(col)])
        axes[i].set_ylabel(f"PC{i+1} load")

    # Set residue names as x-axis tick labels
    axes[-1].set_xticks(range(len(x_labels)))
    axes[-1].set_xticklabels(x_labels, rotation=90)
    axes[0].set_title(figure_title)
    plt.tight_layout()
    return fig, axes

# Perform hierarchical clustering and plot clusters in the PC1–PC2 space
def cluster_and_plot(
    all_trans,
    n_clusters=2,
    highlight_indices=None,
    highlight_color='red',
    main_title="Hierarchical Clustering in PC1–PC2"
):

    hier_clust = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')
    print('ciao')
    print(np.shape(all_trans[:, :2]))
    cluster_labels = hier_clust.fit_predict(all_trans[:, :2])

    bounds = list(range(n_clusters + 1))
    norm_bound = BoundaryNorm(bounds, ncolors=n_clusters, clip=True)

    fig, ax = plt.subplots(figsize=(8, 6))
    scatter = ax.scatter(
        all_trans[:, 0],
        all_trans[:, 1],
        c=cluster_labels,
        cmap='tab10',
        norm=norm_bound,
        alpha=0.8
    )

    if highlight_indices is not None and len(highlight_indices) > 0:
        ax.scatter(
            all_trans[highlight_indices, 0],
            all_trans[highlight_indices, 1],
            facecolors='none',
            edgecolors=highlight_color,
            s=100,
            linewidths=1.5,
            label="Excluded in noOutliers"
        )

    # Add colorbar with cluster labels
    tick_positions = [x + 0.5 for x in range(n_clusters)]
    cbar = plt.colorbar(scatter, spacing="proportional", ticks=tick_positions)
    cbar.ax.set_yticklabels([f"Cluster {i}" for i in range(n_clusters)])
    cbar.set_label("Cluster ID")

    ax.set_xlabel("PC1")
    ax.set_ylabel("PC2")
    ax.set_title(main_title)
    if highlight_indices:
        ax.legend()

    plt.tight_layout()
    return fig, ax, cluster_labels

In [ ]:
"""
This script performs PCA-based analysis and clustering on protein structure datasets from ends-aligned PDB files. It applies both hierarchical and spectral clustering methods to identify structural patterns, while optionally excluding outlier structures. The results, including PCA loadings, projections, and cluster assignments, are visualized and saved to disk. Final clustered structures are copied into designated folders for downstream interpretation.

"""
from glob import glob
import os
import mdtraj as md
# Perform spectral clustering on PCA-transformed data and visualize the results
def spectral_cluster_and_plot(
    all_trans,
    n_clusters=2,
    highlight_indices=None,
    highlight_color='red',
    main_title="Spectral Clustering in PC1–PC2",
    affinity='rbf',
    gamma=1.0,
    explained_var_ratios=None
):
    from sklearn.cluster import SpectralClustering
    from matplotlib.colors import BoundaryNorm
    import matplotlib.pyplot as plt
    import numpy as np
    
    # Fit spectral clustering using PC1 and PC2
    spec_clust = SpectralClustering(
        n_clusters=n_clusters,
        affinity=affinity,
        gamma=gamma,
        random_state=42
    )
    print('Spectral clustering shape:', np.shape(all_trans[:, :2]))
    cluster_labels = spec_clust.fit_predict(all_trans[:, :2])
    
    bounds = list(range(n_clusters + 1))
    norm_bound = BoundaryNorm(bounds, ncolors=n_clusters, clip=True)
    
    # Plot the PCA projection with clusters
    fig, ax = plt.subplots(figsize=(8, 6))
    scatter = ax.scatter(
        all_trans[:, 0],
        all_trans[:, 1],
        c=cluster_labels,
        cmap='tab10',
        norm=norm_bound,
        alpha=0.8
    )

    # Highlight outliers if provided
    if highlight_indices is not None and len(highlight_indices) > 0:
        ax.scatter(
            all_trans[highlight_indices, 0],
            all_trans[highlight_indices, 1],
            facecolors='none',
            edgecolors=highlight_color,
            s=100,
            linewidths=1.5,
            label="Excluded in noOutliers"
        )
    
    # Add colorbar and labels
    tick_positions = [x + 0.5 for x in range(n_clusters)]
    cbar = plt.colorbar(scatter, spacing="proportional", ticks=tick_positions)
    cbar.ax.set_yticklabels([f"Cluster {i}" for i in range(n_clusters)])
    cbar.set_label("Cluster ID")
    
    if explained_var_ratios is not None and len(explained_var_ratios) >= 2:
        ax.set_xlabel(f"PC1 ({explained_var_ratios[0]:.1f}% variance)")
        ax.set_ylabel(f"PC2 ({explained_var_ratios[1]:.1f}% variance)")
    else:
        ax.set_xlabel("PC1")
        ax.set_ylabel("PC2")
    
    ax.set_title(main_title)
    if highlight_indices:
        ax.legend()
    
    plt.tight_layout()
    return fig, ax, cluster_labels

# Main pipeline to run PCA, clustering (hierarchical and spectral), and plotting
def run_ends_alignment_pca_with_spectral(
    all_structures_path,
    no_outliers_path,
    cluster0_output_hier,
    cluster1_output_hier,
    cluster0_output_spec,
    cluster1_output_spec
):

    print(glob(os.path.join(all_structures_path, "*.pdb")))
    all_pdbs = sorted(
        fp for fp in glob(os.path.join(all_structures_path, "*.pdb"))
        if "combined" not in os.path.basename(fp)
    )
    noout_pdbs = sorted(
        fp for fp in glob(os.path.join(no_outliers_path, "*.pdb"))
        if "combined" not in os.path.basename(fp)
    )

    names_all = [pdb_id(fp) for fp in all_pdbs]
    names_no_outliers = [pdb_id(fp) for fp in noout_pdbs]

    print(f"\n======= ENDS ALIGNMENT WITH SPECTRAL CLUSTERING =======")
    print("All structures:", names_all)
    print("No-outlier structures:", names_no_outliers)

    # Identify and index outliers
    excluded_names = set(names_all) - set(names_no_outliers)
    excluded_indices = [i for i, nm in enumerate(names_all) if nm in excluded_names]
    print("Excluded structures (OUTLIERS):", excluded_names)

    # Skip if no files found
    if len(all_pdbs) == 0:
        print(f"WARNING: No PDBs found for ends alignment (All) after excluding 'combined'. Skipping.")
        return

    # Perform PCA on all structures
    all_pca, all_trans = PCA_on_files(all_pdbs, n_components=4)

    explained_var_ratios_all = all_pca.explained_variance_ratio_ * 100
    for i, ratio in enumerate(explained_var_ratios_all, 1):
        print(f"PC{i} (All, Ends Alignment) explains {ratio:.2f}% of variance.")

    # Visualize PCA loading scores
    all_fig, all_axes = plot_scores(
        all_pca,
        n_comps=4,
        folder_for_residues=all_structures_path,
        figure_title="PCA scores - Ends Alignment (ALL)"
    )

    if all_axes is not None:
        for i, ax in enumerate(all_axes.flatten()):
            if hasattr(ax, 'get_xlabel'):
                current_xlabel = ax.get_xlabel()
                if f"PC{i+1}" in current_xlabel and i < len(explained_var_ratios_all):
                    ax.set_xlabel(f"PC{i+1} ({explained_var_ratios_all[i]:.1f}% variance)")
    
    savefile_all_scores = "scores_all_ends.png"
    plt.savefig(savefile_all_scores, bbox_inches='tight', dpi=300)
    plt.close(all_fig)
    
    # Hierarchical clustering visualization
    fig_all, ax_all, cluster_labels_all = cluster_and_plot(
        all_trans,
        n_clusters=2,
        highlight_indices=excluded_indices,
        highlight_color='red',
        main_title="PC1 vs PC2 - All Data (Ends Alignment) - Hierarchical [Red = Outliers]"
    )
    ax_all.set_xlabel(f"PC1 ({explained_var_ratios_all[0]:.1f}% variance)")
    ax_all.set_ylabel(f"PC2 ({explained_var_ratios_all[1]:.1f}% variance)")
    
    cluster_out_all = "cluster_all_ends_hierarchical_with_outliers_highlight.png"
    fig_all.savefig(cluster_out_all, dpi=300)
    plt.close(fig_all)

    # Spectral clustering visualization
    fig_all_spec, ax_all_spec, cluster_labels_all_spec = spectral_cluster_and_plot(
        all_trans,
        n_clusters=2,
        highlight_indices=excluded_indices,
        highlight_color='red',
        main_title="PC1 vs PC2 - All Data (Ends Alignment) - Spectral Clustering [Red = Outliers]",
        explained_var_ratios=explained_var_ratios_all
    )
    cluster_out_all_spec = "cluster_all_ends_spectral_with_outliers_highlight.png"
    fig_all_spec.savefig(cluster_out_all_spec, dpi=300)
    plt.close(fig_all_spec)

    if len(noout_pdbs) == 0:
        print(f"WARNING: No PDBs found for ends alignment (NoOutliers) after excluding 'combined'. Skipping.")
        return

    # Perform PCA on no-outlier subset
    noout_pca, noout_trans = PCA_on_files(noout_pdbs, n_components=4)

    explained_var_ratios_noout = noout_pca.explained_variance_ratio_ * 100
    
    for i, ratio in enumerate(explained_var_ratios_noout, 1):
        print(f"PC{i} (NoOutliers, Ends Alignment) explains {ratio:.2f}% of variance.")

    # Save PC1 and PC2 values with filenames
    out_file1_noout = "pc1_ends_noOutliers.txt"
    print(f"Saving PC1 + filenames for NO-OUTLIERS data to {out_file1_noout}")
    pc1_values = noout_trans[:, 0]
    pc1_names = np.array(names_no_outliers)
    pc1_data = np.column_stack((pc1_values, pc1_names))
    np.savetxt(out_file1_noout, pc1_data, fmt="%s", header="PC1_value,FileName")

    out_file2_noout = "pc2_ends_noOutliers.txt"
    print(f"Saving PC2 + filenames for NO-OUTLIERS data to {out_file2_noout}")
    pc2_values = noout_trans[:, 1]
    pc2_names = np.array(names_no_outliers)
    pc2_data = np.column_stack((pc2_values, pc2_names))
    np.savetxt(out_file2_noout, pc2_data, fmt="%s", header="PC2_value,FileName")

    # Plot PCA scores for no-outlier data
    noout_fig, noout_axes = plot_scores(
        noout_pca,
        n_comps=4,
        folder_for_residues=no_outliers_path,
        figure_title="PCA scores - Ends Alignment (NoOutliers)"
    )
    if noout_axes is not None:
        for i, ax in enumerate(noout_axes.flatten()):
            if hasattr(ax, 'get_xlabel'):
                current_xlabel = ax.get_xlabel()
                if f"PC{i+1}" in current_xlabel and i < len(explained_var_ratios_noout):
                    ax.set_xlabel(f"PC{i+1} ({explained_var_ratios_noout[i]:.1f}% variance)")
    
    savefile_noout_scores = "scores_noOutliers_ends.png"
    plt.savefig(savefile_noout_scores, bbox_inches='tight', dpi=300)
    plt.close(noout_fig)

    # Hierarchical clustering (no outliers)
    fig_noout, ax_noout, cluster_labels_noout = cluster_and_plot(
        noout_trans,
        n_clusters=2,
        main_title="PC1 vs PC2 - No-Outlier Data (Ends Alignment) - Hierarchical"
    )
    ax_noout.set_xlabel(f"PC1 ({explained_var_ratios_noout[0]:.1f}% variance)")
    ax_noout.set_ylabel(f"PC2 ({explained_var_ratios_noout[1]:.1f}% variance)")
    
    cluster_out_noout = "cluster_noOutliers_ends_hierarchical.png"
    fig_noout.savefig(cluster_out_noout, dpi=300)
    plt.close(fig_noout)

    # Spectral clustering (no outliers)
    fig_noout_spec, ax_noout_spec, cluster_labels_noout_spec = spectral_cluster_and_plot(
        noout_trans,
        n_clusters=2,
        main_title="PC1 vs PC2 - No-Outlier Data (Ends Alignment) - Spectral Clustering",
        explained_var_ratios=explained_var_ratios_noout
    )
    cluster_out_noout_spec = "cluster_noOutliers_ends_spectral.png"
    fig_noout_spec.savefig(cluster_out_noout_spec, dpi=300)
    plt.close(fig_noout_spec)

    # Save clustering labels to file
    cluster_labels_file_hier = "cluster_labels_ends_noOutliers_hierarchical.txt"
    cluster_labels_file_spec = "cluster_labels_ends_noOutliers_spectral.txt"
    
    print(f"Saving hierarchical cluster labels + filenames to {cluster_labels_file_hier}")
    print(f"Saving spectral cluster labels + filenames to {cluster_labels_file_spec}")
    
    pdb_codes = [name[:6] for name in names_no_outliers]
    
    cluster_data_hier = np.column_stack((cluster_labels_noout, pdb_codes, names_no_outliers))
    np.savetxt(cluster_labels_file_hier, cluster_data_hier, fmt="%s", header="ClusterLabel,PDBCode,FullName", delimiter=",")

    cluster_data_spec = np.column_stack((cluster_labels_noout_spec, pdb_codes, names_no_outliers))
    np.savetxt(cluster_labels_file_spec, cluster_data_spec, fmt="%s", header="ClusterLabel,PDBCode,FullName", delimiter=",")

    # Copy clustered structures to respective folders
    os.makedirs(cluster0_output_hier, exist_ok=True)
    os.makedirs(cluster1_output_hier, exist_ok=True)
    
    os.makedirs(cluster0_output_spec, exist_ok=True)
    os.makedirs(cluster1_output_spec, exist_ok=True)

    for i, label in enumerate(cluster_labels_noout):
        src_file = noout_pdbs[i]
        if label == 0:
            shutil.copy(src_file, cluster0_output_hier)
        else:
            shutil.copy(src_file, cluster1_output_hier)

    for i, label in enumerate(cluster_labels_noout_spec):
        src_file = noout_pdbs[i]

        if label == 0:
            shutil.copy(src_file, cluster0_output_spec)
        else:
            shutil.copy(src_file, cluster1_output_spec)

    print("Done! Structures have been copied to:")
    print("Hierarchical clustering results:")
    print(f"  Cluster 0 → {cluster0_output_hier}")
    print(f"  Cluster 1 → {cluster1_output_hier}")
    print("Spectral clustering results:")
    print(f"  Cluster 0 → {cluster0_output_spec}")
    print(f"  Cluster 1 → {cluster1_output_spec}")
    print("Both hierarchical and spectral clustering plots have been generated.")

# Entry point for execution
def main_with_spectral():

    all_structures_path = "Results/fitted_matlab_segments/mustang_endsAlignment_cleaned"

    no_outliers_path = "Results/fitted_matlab_segments/mustang_endsAlignment_cleaned_noOutliers/"
    
    cluster0_output_hier = "Results/fitted_matlab_segments/mustangs_ends_noOutlier_pca_clust0_hier"
    cluster1_output_hier = "Results/fitted_matlab_segments/mustangs_ends_noOutlier_pca_clust1_hier"
    cluster0_output_spec = "Results/fitted_matlab_segments/mustangs_ends_noOutlier_pca_clust0_spec"
    cluster1_output_spec = "Results/fitted_matlab_segments/mustangs_ends_noOutlier_pca_clust1_spec"

    run_ends_alignment_pca_with_spectral(all_structures_path, no_outliers_path, 
                                       cluster0_output_hier, cluster1_output_hier,
                                       cluster0_output_spec, cluster1_output_spec)

if __name__ == "__main__":
    main_with_spectral()

['Results/fitted_matlab_segments/mustang_endsAlignment_cleaned/1APM_E.pdb', 'Results/fitted_matlab_segments/mustang_endsAlignment_cleaned/1ATP_E.pdb', 'Results/fitted_matlab_segments/mustang_endsAlignment_cleaned/1BI7_A.pdb', 'Results/fitted_matlab_segments/mustang_endsAlignment_cleaned/1BI8_A.pdb', 'Results/fitted_matlab_segments/mustang_endsAlignment_cleaned/1BI8_C.pdb', 'Results/fitted_matlab_segments/mustang_endsAlignment_cleaned/1BKX_A.pdb', 'Results/fitted_matlab_segments/mustang_endsAlignment_cleaned/1BUH_A.pdb', 'Results/fitted_matlab_segments/mustang_endsAlignment_cleaned/1BX6_A.pdb', 'Results/fitted_matlab_segments/mustang_endsAlignment_cleaned/1CDK_A.pdb', 'Results/fitted_matlab_segments/mustang_endsAlignment_cleaned/1CDK_B.pdb', 'Results/fitted_matlab_segments/mustang_endsAlignment_cleaned/1CM8_A.pdb', 'Results/fitted_matlab_segments/mustang_endsAlignment_cleaned/1CM8_B.pdb', 'Results/fitted_matlab_segments/mustang_endsAlignment_cleaned/1CMK_E.pdb', 'Results/fitted_matlab_s